## Codificación por números enteros

La codificación por enteros consiste en reemplzar las categoriás por dígitos o números enteros del 1 a n ( o de 0 a n-1), dependiendo de la implementación, donde n es el número de las distintas categorias de una variable.

Lo números son asingados arbitrariamente. Este método de codificación permite una rápida evaluación comparativa o 'benchmarking' de los modelos de machine learning.


### Ventajas

- Fácil de implementar
- No expande el espacio de las variables

### Limitaciones

- No añade ninguna información que puede hacer que la  variable tenga mas poder predictivo
- No retiene información sobre las etiquetas de las variables.
- No es adecuado para modelos lineales

La codificación por enteros es más adecuada para metodos no-lineales los cuales pueden navegar sobre los dígitos asignados arbitrariamente para tratar de encontrar los parámetros que pueden relacionarlos con el target o variable objetivo.

## En este demo:

Vamos a realizar codificación one hot con:
- pandas
- Scikit-learn
- Feature-Engine

y las ventajas y limitaciones de cada una de estas implementaciones usando los datos House Prices dataset.


In [34]:
import numpy as np
import pandas as pd

# separar datasets
from sklearn.model_selection import train_test_split

# Codificación por enteros con sklearn
from sklearn.preprocessing import LabelEncoder

# Codificación por enteros con feature-engine
from feature_engine.categorical_encoders import OrdinalCategoricalEncoder

In [35]:
# cargar dataset

data = pd.read_csv(
    '../houseprice.csv',
    usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [36]:
# miremos cuantas etiquetas tiene cada variable

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' etiquetas')

Neighborhood :  25  etiquetas
Exterior1st :  15  etiquetas
Exterior2nd :  16  etiquetas
SalePrice :  663  etiquetas


In [37]:
# exploremos las categorias unicas

data['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [38]:
data['Exterior1st'].unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [39]:
data['Exterior2nd'].unique()

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

### Importante sobre codificación

Es importante seleccionar cual dígito asignar a cada una de las categorias usando el segmento de entrenamiento; y luego usar este mapeo para codificar las variables en el segmento de prueba


In [40]:
# separemos en segmentos de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # predictores
    data['SalePrice'],  # target
    test_size=0.3,  # percentaje observaciones prueba
    random_state=0)  # seed asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

## Codificación por enteros con pandas

### Ventajas

- rápido
- retorna un pandas dataframe

### Limitaciones:

- no preserva la información del segmento de entrenamiento para propagarlo al segmento de prueba

Necesitamos capturar y guardar los mapeos uno por uno, manualmente, si estamos planeando usarlos en producción

In [41]:
# primero creemos un diccionario con los mapeos de las categorias en dígitos o números enteros

ordinal_mapping = {
    k: i
    for i, k in enumerate(X_train['Neighborhood'].unique(), 0)
}

ordinal_mapping

{'CollgCr': 0,
 'ClearCr': 1,
 'BrkSide': 2,
 'Edwards': 3,
 'SWISU': 4,
 'Sawyer': 5,
 'Crawfor': 6,
 'NAmes': 7,
 'Mitchel': 8,
 'Timber': 9,
 'Gilbert': 10,
 'Somerst': 11,
 'MeadowV': 12,
 'OldTown': 13,
 'BrDale': 14,
 'NWAmes': 15,
 'NridgHt': 16,
 'SawyerW': 17,
 'NoRidge': 18,
 'IDOTRR': 19,
 'NPkVill': 20,
 'StoneBr': 21,
 'Blmngtn': 22,
 'Veenker': 23,
 'Blueste': 24}

El diccionario indica cual número va a reemplazar cada categoria. Números fueron asignados arbitrariamente del 0 al n-1 donde n es el número de las diferentes categorias

In [42]:
# reemplza las etiquetas con los enteros


X_train['Neighborhood'] = X_train['Neighborhood'].map(ordinal_mapping)
X_test['Neighborhood'] = X_test['Neighborhood'].map(ordinal_mapping)

In [43]:
# exploremos el resultaod

X_train['Neighborhood'].head(10)

64      0
682     1
960     2
1384    3
1100    4
416     5
1034    6
853     7
472     3
1011    3
Name: Neighborhood, dtype: int64

In [44]:
# podemos convertir los comandos anteriores en dos funciones


def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].unique(), 0)}


def integer_encode(train, test, variable, ordinal_mapping):

    X_train[variable] = X_train[variable].map(ordinal_mapping)
    X_test[variable] = X_test[variable].map(ordinal_mapping)

In [45]:
# y ahora corramos un ciclo sobre el resto de las variables categoricas

for variable in ['Exterior1st', 'Exterior2nd']:
    mappings = find_category_mappings(X_train, variable)
    integer_encode(X_train, X_test, variable, mappings)

In [46]:
# veamos el resultado

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,0,0,0
682,1,1,1
960,2,1,2
1384,3,2,3
1100,4,1,1


## Codificación por enteros con Scikit-learn

In [16]:
# separemos en segmentos de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # percentaje observaciones prueba
    random_state=0)  # seed asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [17]:
# creemos el codificador

le = LabelEncoder()
le.fit(X_train['Neighborhood'])

LabelEncoder()

In [18]:
# veamos las clases únicas

le.classes_

array(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel',
       'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown',
       'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber',
       'Veenker'], dtype=object)

In [19]:
X_train['Neighborhood'] = le.transform(X_train['Neighborhood'])
X_test['Neighborhood'] = le.transform(X_test['Neighborhood'])

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,5,VinylSd,VinylSd
682,4,Wd Sdng,Wd Sdng
960,3,Wd Sdng,Plywood
1384,7,WdShing,Wd Shng
1100,18,Wd Sdng,Wd Sdng


Desafortunadamente, el LabelEncoder solo funciona con una variable SIn embargo hay una forma de automatizarlos para todas las variables categoricas ( al mismo tiempo) a la vez. La implementación a continaución fue tomada de [stackoverflow thread](https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn)

In [20]:
#  import adicional requerido

from collections import defaultdict

In [21]:
# separemos en segmentos de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # percentaje observaciones prueba
    random_state=0)  # seed asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [22]:
d = defaultdict(LabelEncoder)

In [23]:
# codificacieon de la  variable
train_transformed = X_train.apply(lambda x: d[x.name].fit_transform(x))

# # usando el diccionario para codificar datos futuros

test_transformed = X_test.apply(lambda x: d[x.name].transform(x))

In [24]:
train_transformed.head()

,Neighborhood,Exterior1st,Exterior2nd
64,5,12,13
682,4,13,14
960,3,13,10
1384,7,14,15
1100,18,13,14


In [25]:
test_transformed.head()

,Neighborhood,Exterior1st,Exterior2nd
529,6,13,11
491,12,13,14
459,3,8,8
279,4,9,10
655,2,6,7


In [27]:
# y la transformada inversa para recobrar las etiquetas originales

# # Inversa de la codificación

tmp = train_transformed.apply(lambda x: d[x.name].inverse_transform(x))
tmp.head()

,Neighborhood,Exterior1st,Exterior2nd
64,CollgCr,VinylSd,VinylSd
682,ClearCr,Wd Sdng,Wd Sdng
960,BrkSide,Wd Sdng,Plywood
1384,Edwards,WdShing,Wd Shng
1100,SWISU,Wd Sdng,Wd Sdng


Finalmente, hay otro transformador en Scikit-learn, el OrdinalEncoder, para codificar múltiples variables al mismo tiempo. Sin embargo, este transformador retorna un NumPy array sin los nombres de las columnas, por lo tanto no es nuestra implmentacion favorita. Más detalles aqui: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html 

## Codificación por enteros con Feature-Engine

In [28]:
# separemos en segmentos de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # percentaje observaciones prueba
    random_state=0)  # seed asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [29]:
ordinal_enc = OrdinalCategoricalEncoder(
    encoding_method='arbitrary',
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

ordinal_enc.fit(X_train)

OrdinalCategoricalEncoder(encoding_method='arbitrary',
                          variables=['Neighborhood', 'Exterior1st',
                                     'Exterior2nd'])

In [30]:
# en el codificador dict podemos ver los numeros asignados
# a cada categoria para todas las variables indicadas

ordinal_enc.encoder_dict_

{'Neighborhood': {'CollgCr': 0,
  'ClearCr': 1,
  'BrkSide': 2,
  'Edwards': 3,
  'SWISU': 4,
  'Sawyer': 5,
  'Crawfor': 6,
  'NAmes': 7,
  'Mitchel': 8,
  'Timber': 9,
  'Gilbert': 10,
  'Somerst': 11,
  'MeadowV': 12,
  'OldTown': 13,
  'BrDale': 14,
  'NWAmes': 15,
  'NridgHt': 16,
  'SawyerW': 17,
  'NoRidge': 18,
  'IDOTRR': 19,
  'NPkVill': 20,
  'StoneBr': 21,
  'Blmngtn': 22,
  'Veenker': 23,
  'Blueste': 24},
 'Exterior1st': {'VinylSd': 0,
  'Wd Sdng': 1,
  'WdShing': 2,
  'HdBoard': 3,
  'MetalSd': 4,
  'AsphShn': 5,
  'BrkFace': 6,
  'Plywood': 7,
  'CemntBd': 8,
  'Stucco': 9,
  'BrkComm': 10,
  'AsbShng': 11,
  'ImStucc': 12,
  'CBlock': 13,
  'Stone': 14},
 'Exterior2nd': {'VinylSd': 0,
  'Wd Sdng': 1,
  'Plywood': 2,
  'Wd Shng': 3,
  'HdBoard': 4,
  'MetalSd': 5,
  'AsphShn': 6,
  'CmentBd': 7,
  'BrkFace': 8,
  'Stucco': 9,
  'ImStucc': 10,
  'Stone': 11,
  'AsbShng': 12,
  'Brk Cmn': 13,
  'CBlock': 14,
  'Other': 15}}

In [31]:
# esta es la lista de variables que el codificador necesita transformar

ordinal_enc.variables

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [32]:
X_train = ordinal_enc.transform(X_train)
X_test = ordinal_enc.transform(X_test)

# exploremos el resultado
X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,0,0,0
682,1,1,1
960,2,1,2
1384,3,2,3
1100,4,1,1


**Nota**

Si el argmento 'variables' se fija en 'None' (ninguno). entonces el codificador automaticamente indentificará  **todas las variables categóricas**. Maravilloso verdad?

El codificador no codificará las variables numéricas./ Entonces si algunas de tus variables categoricas son de hecho numericas, necesita hacer el 're-cast' o cambio como 'object' antes de usar el codificador.

Si hay una variable en el segmento de prueba, para el cual el codificador no tiene un número para asignar ( la categoria no estaba presente en el segmento de entrenamiento), el codificador devolvera un error.
